# Architecture 3: RAG-Agentique (Agent Augmenté par Outils)

Cette architecture représente l'approche la plus sophistiquée, combinant RAG avec des capacités agentiques selon le paradigme ReAct (Reasoning + Acting).

## Caractéristiques principales:
- **Framework**: LangChain + LangGraph pour orchestration
- **Paradigme**: ReAct (Thought-Action-Observation)
- **Outils**: 4 outils métier (RAG Retriever, Operator Info, Entity Extractor, Conversation Memory)
- **Capacités**: Raisonnement multi-étapes, planification, adaptation contextuelle
- **Base**: Tout ce qui était dans Architecture 2 + couche agentique

<a href="https://colab.research.google.com/github/AmedBah/memoire/blob/main/notebooks/architecture_3/03_architecture_3_rag_agentique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🚀 Configuration pour Google Colab

**Note**: Cette section est spécifique à Google Colab. Si vous exécutez ce notebook localement, vous pouvez ignorer ces cellules.

In [ ]:
# Vérifier le type de runtime
import os
import sys

# Détecter si on est sur Colab
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("✓ Exécution sur Google Colab")
    
    # Vérifier le type de GPU
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✓ GPU détecté: {gpu_name}")
        print(f"✓ Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        
        # Recommandations selon le GPU
        if 'T4' in gpu_name:
            print("\n⚠️  GPU T4 détecté (16 GB): Convient pour ce notebook mais les temps d'entraînement seront plus longs")
        elif 'V100' in gpu_name or 'A100' in gpu_name:
            print(f"\n✓ {gpu_name}: Parfait pour ce notebook!")
    else:
        print("\n❌ ATTENTION: Aucun GPU détecté!")
        print("   Pour activer le GPU: Runtime > Change runtime type > GPU")
        print("   Pour Colab Pro: Choisir 'High-RAM' ou 'Premium GPU'")
else:
    print("✓ Exécution locale")
    import torch
    if torch.cuda.is_available():
        print(f"✓ GPU disponible: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️  Aucun GPU détecté - l'entraînement sera très lent")

In [ ]:
# Monter Google Drive (uniquement sur Colab)
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Définir le chemin vers les données
    # OPTION 1: Données dans Google Drive
    # DATA_DIR = '/content/drive/MyDrive/memoire/data'
    
    # OPTION 2: Cloner le repo et utiliser les données locales
    print("\n📥 Clonage du repository...")
    !git clone https://github.com/AmedBah/memoire.git /content/memoire
    DATA_DIR = '/content/memoire/data'
    
    print(f"\n✓ Répertoire de données: {DATA_DIR}")
    
    # Vérifier que les données sont présentes
    if os.path.exists(DATA_DIR):
        print("✓ Données trouvées!")
        !ls -lh {DATA_DIR}
    else:
        print(f"❌ ERREUR: Répertoire {DATA_DIR} non trouvé!")
        print("   Veuillez soit:")
        print("   1. Copier le dossier 'data' dans votre Google Drive")
        print("   2. Ou le repository sera cloné automatiquement")
else:
    # Exécution locale
    DATA_DIR = '../../data'
    print(f"✓ Répertoire de données local: {DATA_DIR}")

In [ ]:
# Fonction helper pour obtenir les chemins de données
def get_data_path(relative_path):
    """Obtenir le chemin absolu d'un fichier de données"""
    return os.path.join(DATA_DIR, relative_path)

# Exemples de chemins
print("Chemins de données configurés:")
print(f"  Conversations: {get_data_path('conversations/conversation_1000_finetune.jsonl')}")
print(f"  FAQs: {get_data_path('faqs/faq_easytransfert.json')}")
print(f"  Opérateurs: {get_data_path('operators/operators_info.json')}")
print(f"  Procédures: {get_data_path('procedures/procedures_resolution.json')}")
print(f"  Expressions: {get_data_path('expressions/expressions_ivoiriennes.json')}")
print(f"  Documents: {get_data_path('documents/doc.txt.txt')}")

## 1. Installation et Configuration

In [ ]:
# Installation des dépendances
!pip install -q langchain langchain-community langgraph chromadb sentence-transformers transformers torch

import os
import re
import json
from typing import List, Dict, Any, Tuple
from datetime import datetime
import time

import chromadb
from sentence_transformers import SentenceTransformer
from langchain.tools import BaseTool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

print(f"GPU disponible: {torch.cuda.is_available()}")

## 2. Initialisation ChromaDB et Embedding (identique Architecture 2)

In [ ]:
# Réutiliser la base ChromaDB de l'Architecture 2
chroma_client = chromadb.PersistentClient(path="./chromadb_easytransfert")
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

collection = chroma_client.get_or_create_collection(
    name="easytransfert_knowledge_base",
    metadata={"description": "Base de connaissances EasyTransfert pour RAG"}
)

print(f"Collection chargée: {collection.count()} documents")

## 3. Outil 1: RAG Retriever

Encapsulation de la recherche vectorielle comme outil invocable

In [ ]:
from langchain.tools import Tool

class RAGRetrieverTool(BaseTool):
    name: str = "rag_retriever"
    description: str = """Recherche d'informations dans la base de connaissances EasyTransfert.
    Utilise ce tool pour trouver des informations sur:
    - FAQ et questions fréquentes
    - Procédures de résolution de problèmes
    - Documentation des opérateurs
    - Historique de conversations
    
    Entrée: Une question ou requête en français
    Sortie: Liste de documents pertinents avec métadonnées
    """
    
    def _run(self, query: str, top_k: int = 3) -> str:
        """Recherche synchrone"""
        query_embedding = embedding_model.encode([query])[0]
        
        results = collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k,
        )
        
        # Formater les résultats
        formatted_results = []
        for i in range(len(results['documents'][0])):
            distance = results['distances'][0][i]
            similarity = 1 - distance
            
            formatted_results.append({
                "content": results['documents'][0][i],
                "category": results['metadatas'][0][i].get('category', 'unknown'),
                "source": results['metadatas'][0][i].get('source', 'interne'),
                "score": round(similarity, 3)
            })
        
        return json.dumps(formatted_results, ensure_ascii=False, indent=2)
    
    async def _arun(self, query: str) -> str:
        """Recherche asynchrone"""
        return self._run(query)

rag_tool = RAGRetrieverTool()
print("Outil RAG Retriever créé")

## 4. Outil 2: Operator Info

Consultation d'informations structurées sur les opérateurs

In [ ]:
class OperatorInfoTool(BaseTool):
    name: str = "operator_info"
    description: str = """Consulte les informations structurées sur les opérateurs mobiles.
    Fournit des données techniques:
    - Formats d'identifiants de transaction
    - Limites min/max de transfert
    - Frais de transaction
    - Préfixes de numéros de téléphone
    - Compatibilités inter-opérateurs
    
    Entrée: Nom de l'opérateur (MTN, Orange, Moov, Wave, Trésor Money) ou 'tous'
    Sortie: Informations détaillées sur l'opérateur
    """
    
    # Base de données fictive des opérateurs
    operators_db = {
        "MTN": {
            "nom_complet": "MTN Mobile Money",
            "format_id": "Chiffres uniquement (généralement 10 chiffres)",
            "exemple_id": "1234567890",
            "limite_min": "100 FCFA",
            "limite_max": "2 000 000 FCFA",
            "frais": "1-2% (min 25 FCFA, max 500 FCFA)",
            "prefixes": ["05", "06", "07"],
            "compatible_avec": ["Orange", "Moov", "Wave", "Trésor Money"]
        },
        "Orange": {
            "nom_complet": "Orange Money",
            "format_id": "MP suivi de 10 chiffres (envoi)",
            "exemple_id": "MP1234567890",
            "limite_min": "100 FCFA",
            "limite_max": "1 500 000 FCFA",
            "frais": "1-2% (min 25 FCFA, max 500 FCFA)",
            "prefixes": ["07", "08", "09"],
            "compatible_avec": ["MTN", "Moov", "Wave", "Trésor Money"]
        },
        "Moov": {
            "nom_complet": "Moov Money",
            "format_id": "MRCH* ou CF* + alphanumériques (envoi)",
            "exemple_id": "MRCH123456 ou CF789012",
            "limite_min": "100 FCFA",
            "limite_max": "1 000 000 FCFA",
            "frais": "1-2% (min 25 FCFA, max 500 FCFA)",
            "prefixes": ["01", "02"],
            "compatible_avec": ["MTN", "Orange", "Wave", "Trésor Money"]
        },
        "Wave": {
            "nom_complet": "Wave",
            "format_id": "Format variable, souvent T + chiffres",
            "exemple_id": "T123456789",
            "limite_min": "100 FCFA",
            "limite_max": "5 000 000 FCFA",
            "frais": "1% (min 25 FCFA, max 500 FCFA)",
            "prefixes": ["05", "07"],
            "compatible_avec": ["MTN", "Orange", "Moov", "Trésor Money"]
        },
        "Trésor Money": {
            "nom_complet": "Trésor Money (Trémo)",
            "format_id": "Format variable selon transaction",
            "exemple_id": "Variable",
            "limite_min": "100 FCFA",
            "limite_max": "1 000 000 FCFA",
            "frais": "1-2% (min 25 FCFA, max 500 FCFA)",
            "prefixes": ["05"],
            "compatible_avec": ["MTN", "Orange", "Moov", "Wave"]
        }
    }
    
    def _run(self, operator_name: str) -> str:
        """Recherche synchrone"""
        operator_name = operator_name.strip().title()
        
        if operator_name.lower() == "tous" or operator_name.lower() == "all":
            return json.dumps(self.operators_db, ensure_ascii=False, indent=2)
        
        # Chercher l'opérateur
        for key in self.operators_db:
            if operator_name in key or key in operator_name:
                return json.dumps({key: self.operators_db[key]}, ensure_ascii=False, indent=2)
        
        return json.dumps({"error": f"Opérateur '{operator_name}' non trouvé. Opérateurs disponibles: MTN, Orange, Moov, Wave, Trésor Money"}, ensure_ascii=False)
    
    async def _arun(self, operator_name: str) -> str:
        return self._run(operator_name)

operator_tool = OperatorInfoTool()
print("Outil Operator Info créé")

## 5. Outil 3: Entity Extractor

Extraction d'entités nommées via regex et règles métier

In [ ]:
class EntityExtractorTool(BaseTool):
    name: str = "entity_extractor"
    description: str = """Extrait des entités structurées d'un texte utilisateur.
    Identifie:
    - Identifiants EasyTransfert (EFB.*)
    - Identifiants opérateurs (MTN, Orange MP*, Moov MRCH*/CF*, Wave T*)
    - Numéros de téléphone
    - Montants en FCFA
    - Noms d'opérateurs mentionnés
    
    Entrée: Texte contenant potentiellement des entités
    Sortie: Dictionnaire des entités extraites par catégorie
    """
    
    patterns = {
        "easytransfert_id": r"EFB\.[A-Z0-9]+",
        "mtn_id": r"\b\d{10,}\b",
        "orange_id": r"MP\d{10}",
        "moov_id": r"(MRCH|CF)[A-Z0-9]+",
        "wave_id": r"T\d+",
        "phone": r"(\+225)?\s?[0-9]{8,10}",
        "amount": r"\d+\s*(FCFA|CFA|francs?|F)?",
        "operators": r"(MTN|Orange|Moov|Wave|Trésor\s?Money|Tremo)"
    }
    
    def _run(self, text: str) -> str:
        """Extraction synchrone"""
        extracted = {}
        
        for entity_type, pattern in self.patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                extracted[entity_type] = list(set(matches))  # Dédupliquation
        
        if not extracted:
            return json.dumps({"message": "Aucune entité détectée"}, ensure_ascii=False)
        
        return json.dumps(extracted, ensure_ascii=False, indent=2)
    
    async def _arun(self, text: str) -> str:
        return self._run(text)

entity_tool = EntityExtractorTool()
print("Outil Entity Extractor créé")

# Test
test_text = "Mon transfert EFB.ABC123 de 50000 FCFA vers MTN n'est pas arrivé. Mon numéro est 0758123456"
print(f"\nTest extraction: {test_text}")
print(entity_tool._run(test_text))

## 6. Outil 4: Conversation Memory

Gestion de l'historique conversationnel

In [ ]:
class ConversationMemoryTool(BaseTool):
    name: str = "conversation_memory"
    description: str = """Gère l'historique de la conversation avec l'utilisateur.
    Permet de:
    - Récupérer le contexte de la conversation actuelle
    - Rechercher des problèmes similaires passés
    - Éviter de redemander des informations déjà fournies
    
    Entrée: 'get_history' pour récupérer l'historique ou une requête pour chercher
    Sortie: Historique ou résultats de recherche
    """
    
    # Stockage en mémoire (dans une vraie implémentation, utiliser une base de données)
    memory_store: Dict[str, List[Dict]] = {}
    
    def _run(self, query: str, user_id: str = "default") -> str:
        """Opération synchrone"""
        
        if user_id not in self.memory_store:
            self.memory_store[user_id] = []
        
        if query.lower() == "get_history":
            return json.dumps({
                "history": self.memory_store[user_id],
                "count": len(self.memory_store[user_id])
            }, ensure_ascii=False, indent=2)
        
        # Chercher dans l'historique
        relevant = []
        for entry in self.memory_store[user_id]:
            if any(word.lower() in entry.get("content", "").lower() for word in query.split()):
                relevant.append(entry)
        
        return json.dumps({
            "relevant_entries": relevant,
            "count": len(relevant)
        }, ensure_ascii=False, indent=2)
    
    async def _arun(self, query: str, user_id: str = "default") -> str:
        return self._run(query, user_id)
    
    def add_to_memory(self, user_id: str, role: str, content: str):
        """Ajoute un message à la mémoire"""
        if user_id not in self.memory_store:
            self.memory_store[user_id] = []
        
        self.memory_store[user_id].append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })

memory_tool = ConversationMemoryTool()
print("Outil Conversation Memory créé")

## 7. Configuration du LLM et de l'Agent ReAct

In [ ]:
# Charger le modèle LLM
model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

# Créer le pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
)

llm = HuggingFacePipeline(pipeline=pipe)

print("LLM chargé avec succès")

## 8. Prompt ReAct pour l'Agent

Template structurant le cycle Thought-Action-Observation

In [ ]:
react_prompt_template = """Tu es un agent intelligent du service client EasyTransfert.
Tu as accès aux outils suivants pour aider les utilisateurs:

{tools}

PARADIGME ReAct - Cycle Pensée-Action-Observation:

Pour chaque requête utilisateur, suis ce processus itératif:

1. Thought (Pensée): Analyse la situation et planifie la prochaine étape
2. Action: Choisis un outil et spécifie l'entrée
3. Observation: Examine le résultat de l'action
4. Répète jusqu'à avoir assez d'informations pour répondre

FORMAT DE RÉPONSE:

Thought: [ton raisonnement sur ce qu'il faut faire]
Action: [nom de l'outil à utiliser]
Action Input: [entrée pour l'outil]
Observation: [résultat retourné par l'outil]
... (répéter Thought/Action/Observation autant que nécessaire)
Thought: J'ai maintenant assez d'informations pour répondre
Final Answer: [ta réponse finale à l'utilisateur]

RÈGLES IMPORTANTES:
- Utilise les outils de manière stratégique
- Pour les problèmes de transaction, extrais d'abord les entités puis cherche dans la base
- Cite toujours tes sources quand tu utilises rag_retriever
- Vérifie la mémoire pour éviter de redemander des informations
- Ton chaleureux avec émojis 🤗😊
- En cas de doute, référence au service client: 2522018730

Commence maintenant!

Question: {input}

{agent_scratchpad}
"""

react_prompt = PromptTemplate(
    template=react_prompt_template,
    input_variables=["tools", "input", "agent_scratchpad"]
)

print("Prompt ReAct défini")

## 9. Création de l'Agent ReAct

In [ ]:
# Liste des outils disponibles
tools = [
    rag_tool,
    operator_tool,
    entity_tool,
    memory_tool
]

# Créer l'agent ReAct
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt
)

# Créer l'executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True,
)

print("Agent ReAct créé et prêt!")
print(f"Nombre d'outils disponibles: {len(tools)}")

## 10. Fonction de Chat Agentique

In [ ]:
def agentic_chat(query: str, user_id: str = "default") -> Dict:
    """
    Chat avec l'agent ReAct
    
    Args:
        query: Question de l'utilisateur
        user_id: Identifiant utilisateur pour la mémoire
    
    Returns:
        Dict avec réponse, outils utilisés et métriques
    """
    start_time = time.time()
    
    # Ajouter à la mémoire
    memory_tool.add_to_memory(user_id, "user", query)
    
    # Exécuter l'agent
    try:
        result = agent_executor.invoke({"input": query})
        response = result.get("output", "Je n'ai pas pu générer une réponse. Contactez le 2522018730 🤗")
        
        # Ajouter la réponse à la mémoire
        memory_tool.add_to_memory(user_id, "assistant", response)
        
    except Exception as e:
        response = f"Une erreur s'est produite: {str(e)}. Contactez le service client au 2522018730 🤗"
    
    total_time = time.time() - start_time
    
    return {
        "response": response,
        "execution_time_ms": total_time * 1000,
        "user_id": user_id
    }

print("Fonction de chat agentique définie")

## 11. Tests de l'Agent ReAct

In [ ]:
# Tests progressifs
test_scenarios = [
    {
        "name": "Question FAQ simple",
        "query": "Quels sont les opérateurs supportés par EasyTransfert ?"
    },
    {
        "name": "Extraction d'entités + recherche opérateur",
        "query": "Quel est le format d'identifiant pour Orange Money ?"
    },
    {
        "name": "Problème complexe avec extraction",
        "query": "Mon transfert EFB.ABC123 de 50000 FCFA vers MTN n'est pas arrivé"
    },
    {
        "name": "Multi-étapes avec mémoire",
        "query": "Je veux vérifier les limites de transfert pour l'opérateur que j'ai mentionné"
    },
]

print("=" * 80)
print("TESTS DE L'AGENT ReAct")
print("=" * 80)

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n{'='*80}")
    print(f"Test {i}: {scenario['name']}")
    print(f"{'='*80}")
    print(f"Question: {scenario['query']}\n")
    
    result = agentic_chat(scenario['query'])
    
    print(f"\nRéponse: {result['response']}")
    print(f"\nTemps d'exécution: {result['execution_time_ms']:.0f}ms")
    print("=" * 80)

## 12. Analyse des Performances

In [ ]:
import numpy as np

# Métriques de performance
execution_times = []

test_queries = [
    "Comment faire un transfert ?",
    "Quels sont les frais ?",
    "Mon argent n'est pas arrivé",
    "Format identifiant Wave ?",
    "Limite de transfert MTN ?"
]

for query in test_queries:
    result = agentic_chat(query)
    execution_times.append(result['execution_time_ms'])

print("\nMÉTRIQUES DE PERFORMANCE - ARCHITECTURE 3 (RAG-Agentique)")
print("=" * 80)
print(f"Temps d'exécution:")
print(f"  - Moyenne: {np.mean(execution_times):.0f}ms")
print(f"  - Médiane: {np.median(execution_times):.0f}ms")
print(f"  - Min/Max: {np.min(execution_times):.0f}ms / {np.max(execution_times):.0f}ms")
print(f"  - Écart-type: {np.std(execution_times):.0f}ms")
print(f"\nOutils disponibles: {len(tools)}")
print(f"Capacités agentiques: Raisonnement multi-étapes, utilisation d'outils, mémoire contextuelle")
print("=" * 80)

## 13. Comparaison des 3 Architectures

### Architecture 1 (Baseline - Fine-tuning):
- ⚡ **Latence**: ~2-3s
- 💾 **Mémoire**: Faible (~50 MB adaptateurs LoRA)
- 🎯 **Complexité**: Très simple
- ❌ **Hallucinations**: Risque élevé
- ❌ **Actualisation**: Nécessite réentraînement
- ❌ **Traçabilité**: Aucune

### Architecture 2 (RAG Standard):
- ⚡ **Latence**: ~2-3.5s (récupération + génération)
- 💾 **Mémoire**: Moyenne (ChromaDB + LLM)
- 🎯 **Complexité**: Modérée
- ✅ **Hallucinations**: Fortement réduit
- ✅ **Actualisation**: Ajout de documents sans réentraînement
- ✅ **Traçabilité**: Citations des sources
- ❌ **Raisonnement**: Limité aux requêtes simples

### Architecture 3 (RAG-Agentique):
- ⚡ **Latence**: ~3-5s (cycle ReAct itératif)
- 💾 **Mémoire**: Élevée (ChromaDB + LLM + outils)
- 🎯 **Complexité**: Élevée
- ✅ **Hallucinations**: Minimisé (validation par outils)
- ✅ **Actualisation**: Mise à jour des outils et base
- ✅ **Traçabilité**: Complète (cycle ReAct visible)
- ✅ **Raisonnement**: Multi-étapes et planification
- ✅ **Outils**: Accès bases de données, APIs externes
- ✅ **Adaptation**: Contextuelle et émotionnelle

### Conclusion:
L'Architecture 3 représente la solution la plus complète pour un service client automatisé intelligent. Elle combine:
- La fiabilité du RAG (Architecture 2)
- L'autonomie décisionnelle des agents
- L'accès à des outils métier spécialisés
- La capacité de raisonnement multi-étapes

Le surcoût en latence (~1-2s) est justifié par la qualité et la pertinence accrues des réponses.